In [1]:
# from data_gen import *
from models import *
from heb_proc import *

In [2]:
name = "turkish"
epochs = 54

In [3]:

featMap, root2feat, root2word = read(file='data/sig/{0}-train.txt'.format(name))
char2int, int2char, max_root, max_word, feat_length = stat(root2feat, root2word, featMap)
print(max_word, max_root, feat_length)
roots_mat, decoder_words_mat, feats_vec, out_words_mat = get_data(root2word, root2feat, char2int, max_root, max_word, feat_length)
batch_size = 64
n_input_length = roots_mat.shape[1]
n_output_shape = out_words_mat.shape[1]
n_feature = feat_length
n_batches = len(root2feat) // batch_size
print("Root shape: ", roots_mat.shape)
print("Decoder input shape: ", decoder_words_mat.shape)
print("Feature shape: ", feats_vec.shape)
print("Output shape: ", out_words_mat.shape)


26 24 41
Root shape:  (12029, 24, 34)
Decoder input shape:  (12029, 28, 34)
Feature shape:  (12029, 41)
Output shape:  (12029, 28, 34)


In [4]:
gen = gen_data(roots_mat, decoder_words_mat, feats_vec, out_words_mat)
# (roots, decode, feat), words = next(gen)

In [5]:
# words.shape

In [6]:
# gen = gen_data(roots_mat, words_mat, feats_vec, batch_size=batch_size)

In [7]:
root_h, root_w = roots_mat.shape[1], roots_mat.shape[2]
decoder_input = decoder_words_mat.shape[2]
decoder_output = out_words_mat.shape[1]
feat_length = feats_vec.shape[1]
model, encoder, decoder = conv_model(root_h, root_w, decoder_input, decoder_output, feat_length, 256, feat_units=15)
model.compile(optimizer='adam', loss='categorical_crossentropy')


In [8]:
# model.summary()
# arabic 50

In [9]:
# model train given the data generator, how many batches and number of epochs
history = model.fit_generator(gen, steps_per_epoch=n_batches, epochs = epochs)

Epoch 1/54
187/187 [==============================] - 10s 55ms/step - loss: 1.2003
Epoch 2/54
187/187 [==============================] - 9s 46ms/step - loss: 0.5131
Epoch 3/54
187/187 [==============================] - 9s 45ms/step - loss: 0.2892
Epoch 4/54
187/187 [==============================] - 8s 43ms/step - loss: 0.1830
Epoch 5/54
187/187 [==============================] - 8s 42ms/step - loss: 0.1268
Epoch 6/54
187/187 [==============================] - 8s 42ms/step - loss: 0.0947
Epoch 7/54
187/187 [==============================] - 8s 42ms/step - loss: 0.0707
Epoch 8/54
187/187 [==============================] - 8s 42ms/step - loss: 0.0547
Epoch 9/54
187/187 [==============================] - 8s 42ms/step - loss: 0.0447
Epoch 10/54
187/187 [==============================] - 8s 41ms/step - loss: 0.0324
Epoch 11/54
187/187 [==============================] - 8s 42ms/step - loss: 0.0267
Epoch 12/54
187/187 [==============================] - 8s 41ms/step - loss: 0.0217
Epoch 13/54


In [10]:
tfeat_map, troot2feat, troot2word = read('data/sig/{0}-train.txt'.format(name), featMap)
troots_mat, tdecoder_words_mat, tfeats_vec, tout_words_mat = get_data(troot2word, troot2feat, char2int, max_root, max_word, feat_length)
test_gen = gen_data(troots_mat, tdecoder_words_mat, tfeats_vec, tout_words_mat)

In [11]:
def predict(encoder, decoder, root_word, word_feat, max_step, output_dim, char2int):
    # encode
    state = encoder.predict([root_word, word_feat])
    # start of sequence input
    start = [0.0 for _ in range(output_dim)]
    start[char2int['&']] = 1
    target_seq = np.array(start).reshape(1, 1, output_dim)
    # collect predictions
    output = list()
    for t in range(max_step):
        # predict next char
        yhat, h= decoder.predict([target_seq, state])
        # store prediction
        output.append(yhat[0,0,:])
        # update state
        state = h
        # update target sequence
        target_seq = yhat
    return np.array(output)

In [12]:
test_n_batches = len(troot2feat) // batch_size

In [13]:
# "ll" is "l"

In [14]:
# shows sample examples and calculates accuracy

total, correct = 0, 0
in_word = 0
sims = []
for b in range(test_n_batches):
    # get data from test data generator
    [roots, feats, words], y = next(test_gen)
    for j in range(batch_size):
        word_features = feats[j].reshape((1, feats.shape[1])) 
        root_word_matrix = roots[j].reshape((1, roots.shape[1], roots.shape[2], 1))
        
        # predicts the target word given root word and features
        target = predict(encoder, decoder, root_word_matrix, word_features, max_word, decoder_input, char2int)
        root = ''.join(one_hot_decode(roots[j], int2char)).replace('&', ' ').strip()
        word = ''.join(one_hot_decode(y[j], int2char)).replace('&', ' ').strip()
        targetS = ''.join(one_hot_decode(target, int2char)).replace('&', ' ').strip()
#         print(word, len(word), targetS, len(targetS))
#         sims.append(dg.word_sim(word, targetS))
        target =  target[:-1]
        # checks if the predicted and the real words are equal
        
        if word == targetS:
            correct += 1
#         else:
#             print(root, word.split('&')[0], '\t\t', targetS.split('&')[0])
#         if root.strip() in targetS.strip():
#             in_word += 1
#     print(b, root, word, targetS)
    total += batch_size
    

print('Exact Accuracy: %.2f%%' % (float(correct)/float(total)*100.0))

Exact Accuracy: 99.99%


In [15]:
#arabic 50 99.94
#finnish 50 96.09
#georgian 54 99.99
#german 54 99.99%
# hungary 30 99.97
# navajo 99.97%
# russian 100%
# spanish 99.98%
# turkish 99.98%
